## Azure Machine Learning での実験

### Azure Machine Learning への対応

#### ライブラリ
conda yml ファイルにおいて、Azure Machine Learning の Python SDK `azureml-sdk` は必ず含むようにします。

#### データセット (引数で引き渡す)
学習データは Azure Machine Learning Dataset (or Datastore)に登録されているものを利用します。`ScriptRunConfig` に `dataset.as_named_input('input').as_mount()` のように、 Dataset の情報、名称(エイリアス)、下記いずれかの提供方法についての情報をインプットします。
- `as_mount()` : ファイルをマウントする
- `as_download()` : ファイルをダウンロードする
    
#### メトリック記録
モデル学習において、ハイパーパラメータやモデル精度などのメトリックを記録することができます。各実験の記録を横串で可視化して比較することもできます。
- `run = Run.get_context()` : run オブジェクトの生成
- `run.log("xxx", xxx)`  : 実験のメトリックとして記録

In [1]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.widgets import RunDetails

In [2]:
ws = Workspace.from_config()

### Database の取得

In [3]:
dataset = Dataset.get_by_name(ws, name='cifar10')

### 実験名の設定

In [4]:
experiment = Experiment(workspace=ws, name='3-aml-run')

### 学習スクリプトの実行設定

In [ ]:

config = ScriptRunConfig(source_directory='./code/pytorch-cloud',
                         script='train.py',
                         compute_target='local',
                         arguments=[
                            '--data_path', dataset.as_named_input('input').as_download(),
                            '--learning_rate', 0.003,
                            '--momentum', 0.92])

### Environment の呼び出しと実行設定へのインプット

In [6]:
env = Environment.get(ws, "pytorch-env")

In [7]:
config.run_config.environment = env

### 実行と結果確認

In [8]:
run = experiment.submit(config)
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

**参考情報**
- [トレーニングの実行を構成して送信する](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-set-up-training-targets)